## 01. Load mgatk variants (and filter if needed)

In [1]:
indir = "/home/isshamie/data/Projects/Mito_Trace/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/clones_init/donor0"
name = "donor0"
donor = 0
outdir = "/home/isshamie/data/Projects/Mito_Trace/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_10/donor0"


# Cluster parameters
kparam = 10 

cells_col <- "donor"
cell_id_col <- "ID"




### Load Libraries

In [2]:
library(data.table)
library(dplyr)
library(SummarizedExperiment)
library(Matrix)
library(BuenColors)

library(future)
plan()
#plan("multiprocess", workers = workers)
options(future.globals.maxSize = 8000 * 1024^2)

library(Seurat)
library(Signac)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeigh

sequential:
- args: function (..., envir = parent.frame())
- tweaked: FALSE
- call: NULL

Attaching SeuratObject


Attaching package: ‘Seurat’


The following object is masked from ‘package:SummarizedExperiment’:

    Assays




In [3]:
# SEall <- readRDS(indir) #"/data2/mito_lineage/data/processed/mttrace/TcellDupi_may17_2021/MTblacklist/pre/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/pre.signac.rds")
# SEall

## Subset on cells and variants if given

In [4]:
af_ind = file.path(indir, "af.tsv")

In [5]:
af.vals = t(read.table(af_ind,header=T, row.names=1, check.names=F))
print(head(af.vals))


       AAACGAAAGTCGTGAG-1_Control AAACGAACACAATAAG-1_Control
10086G                  0.2222222                  0.0000000
10318C                  0.0000000                  0.0000000
10373A                  0.0000000                  0.0000000
10398G                  1.0000000                  0.9534884
103A                    0.0000000                  0.0000000
10400T                  1.0000000                  0.9523810
       AAACGAAGTAAGTCTC-1_Control AAACGAAGTGGATTCT-1_Control
10086G                          0                          0
10318C                          0                          0
10373A                          0                          0
10398G                          1                          1
103A                            0                          0
10400T                          1                          1
       AAACGAATCGCAACTA-1_Control AAACTCGCAAGACTTC-1_Control
10086G                 0.06060606                          0
10318C                 0

In [6]:
cell.meta <- data.frame(row.names=colnames(af.vals), condition=gsub("^.*_", "",colnames(af.vals)),
                       stringsAsFactors=T)
cell.meta

,condition
,<fct>
AAACGAAAGTCGTGAG-1_Control,Control
AAACGAACACAATAAG-1_Control,Control
AAACGAAGTAAGTCTC-1_Control,Control
AAACGAAGTGGATTCT-1_Control,Control
AAACGAATCGCAACTA-1_Control,Control
AAACTCGCAAGACTTC-1_Control,Control
AAACTCGCATCGCCTT-1_Control,Control
AAACTCGGTATCCTTT-1_Control,Control
AAACTCGTCGAGTGTT-1_Control,Control


In [7]:
nrows <- 200
ncols <- 6
counts <- matrix(runif(nrows * ncols, 1, 1e4), nrows)
rowRanges <- GRanges(rep(c("chr1", "chr2"), c(50, 150)),
                     IRanges(floor(runif(200, 1e5, 1e6)), width=100),
                     strand=sample(c("+", "-"), 200, TRUE),
                     feature_id=sprintf("ID%03d", 1:200))
colData <- DataFrame(Treatment=rep(c("ChIP", "Input"), 3),
                     row.names=LETTERS[1:6])
colData

DataFrame with 6 rows and 1 column
    Treatment
  <character>
A        ChIP
B       Input
C        ChIP
D       Input
E        ChIP
F       Input

In [8]:
assay(SummarizedExperiment(assays=list(counts=counts),
                     rowRanges=rowRanges, colData=colData))

A,B,C,D,E,F
5171.3062,327.2045,4752.301,7680.1293,6853.2296,4686.9558
4335.8708,4739.7920,1734.665,6054.6999,3829.0628,878.4568
9337.2281,3409.7823,4769.295,4983.0119,4378.2400,6485.0691
1951.3008,6998.5558,3109.591,6658.9752,9284.6598,884.2394
2500.2655,5649.7202,9989.615,4824.6521,8432.6535,271.4877
7307.4505,6964.8999,5321.615,2110.3458,8513.5893,4028.7572
8743.2317,1090.2491,2103.176,4033.5001,1471.4659,4576.1892
7871.8512,7078.2207,7656.916,4861.2575,7259.9677,2602.7109
6821.3811,1194.2213,5669.806,9903.9964,3988.5034,5117.6960
109.9352,4896.9693,6808.767,341.2608,740.1157,6841.9223


In [9]:
SE <- SummarizedExperiment(assays=list(allele_frquency=af.vals), colData = cell.meta)
print(dim(assay(SE)))

[1]  220 4205


## 02. Cluster using Seurat FindNeighbors (S-NN graph ) and FindClusters (louvain)

In [10]:
getNN <- function(mat_af, k.param = 10){
  set.seed(1)
  rownames(mat_af) <- make.unique(rownames(mat_af))
  obj <- FindNeighbors(mat_af, k.param = k.param, annoy.metric = "cosine")
  obj
}

# Function to use Seurat functions to call mitochondrial cluster clones
seuratSNN_cosineDistance <- function(obj, resolution){
  clusters <- FindClusters(object = obj$snn, resolution = resolution)
  return(as.character(clusters[,1]))
}


In [11]:
plot_af_clusters <- function(afin, cluster_name){
    mdf <- data.frame(cluster_name, t(afin), check.names=F) %>%
    group_by(cluster_name) %>% dplyr::filter(n() >= 5) %>% summarize_all(.funs = mean)

    melt_df <- reshape2::melt(mdf, id.vars = "cluster_name")

    # This series of arrangements is for aesthetics
    melt_df2 <- melt_df %>% mutate(g5 = value >= 0.05, g1 = value >= 0.01, sqrt = sqrt(value)) %>%  group_by(variable) %>%
    mutate(total_g1 = sum(g1)) %>% mutate(total_g5 = sum(g5)) %>% 
    arrange(desc((g5)), desc((g1))) %>% dplyr::filter(total_g1 > 0) 

    # More aesthetics
    melt_df2$cluster_name <- factor(as.character(melt_df2$cluster_name), levels = rev(unique(as.character(melt_df2$cluster_name))))
    melt_df2$variable <- factor(as.character(melt_df2$variable ), levels = (unique( as.character(melt_df2$variable))))
    melt_df2$value <- ifelse(melt_df2$value > 0.05, 1, melt_df2$value)
    melt_df2$value <- ifelse(melt_df2$value < 0.005, 0, melt_df2$value)
    

    # Visualize heatmap of clusters x variants number cells > 0.01
    p.numvar1 <- ggplot(melt_df2 , aes(x = variable, y = cluster_name, fill = total_g5)) +
    geom_tile() +
    scale_fill_gradientn(colors = jdb_palette("solar_rojos")) +
    theme(axis.text.x=element_blank(),
          axis.ticks.x=element_blank()) +
    theme(axis.title.y=element_blank(),
          axis.text.y=element_blank(),
          axis.ticks.y=element_blank()) +
    xlab("Variant") +
    ggtitle("Total number of cells greater than 0.05 in variant") +
    L_border() #+
    #theme(legend.position = "none")
    ggsave(file.path(outdir, paste0(name,".variants.g5.png")))
    #ggsave(file.path(outdir, "variants.png"))
    #cowplot::ggsave(p1, file = paste0("../plots/mutations_clones_",what,"_invivo_grid.pdf"), width = 3.2, height = 1.5)
    print(length(unique(melt_df2$variable)))
    print(length(unique(melt_df2$cluster_name)))

    # Visualize heatmap of clusters x variants
    p.bin <- ggplot(melt_df2 , aes(x = variable, y = cluster_name, fill = value)) +
    geom_tile() +
    scale_fill_gradientn(colors = jdb_palette("solar_rojos")) +
    theme(axis.text.x=element_blank(),
          axis.ticks.x=element_blank()) +
    theme(axis.text.y=element_blank(),
          axis.ticks.y=element_blank()) +
    xlab("Variant") +
    ylab("Cluster") +
    ggtitle("Variant avg greater than 0.05 set to 1 and <0.005 to 0") +
    L_border() #+
           
    #theme(legend.position = "none")
    print("Number of variants")
    print(length(unique(melt_df2$variable)))
    print("Number of clusters")
    print(length(unique(melt_df2$cluster_name)))
    ggsave(file.path(outdir, paste0(name, ".variants.binary.png")))
    
    p.varmean <- ggplot(melt_df2 , aes(x = variable, y = cluster_name, fill = sqrt)) +
    geom_tile() +
    scale_fill_gradientn(colors = jdb_palette("solar_rojos")) +
    theme(axis.text.x=element_blank(),
          axis.ticks.x=element_blank()) +
    theme(axis.title.y=element_text("Cluster"),
          axis.text.y=element_blank(),
          axis.ticks.y=element_blank()) +
    xlab("Variant") +
    ylab("Cluster") +
    ggtitle("Sqrt Mean AF in each cluster") +
    L_border() #+
    ggsave(file.path(outdir, paste0(name,".variants.labels.png")))
    #theme(legend.position = "none")

    return(c(melt_df2, p.numvar1, p.varmean))
}

In [12]:
run_clones <- function(SE, curr_out, k.param){
    afin <- data.matrix(assay(SE))  
    #print('afin')
    #print(head(afin))
    #print(head(rownames(afin)))
    # Since the PBMCs alone undercalled the mutations, we will examine the mutations called in the PBMCs and CD34s together
    obj <- getNN(t(sqrt(afin)), k.param=k.param)
    clusters <- seuratSNN_cosineDistance(obj, resolution = 3.5) 
    cluster_name <- stringr::str_pad(as.character(clusters), 3, pad = "0")
    table(cluster_name)
    saveRDS(clusters, file.path(curr_out, "clusters.rds"))
    saveRDS(obj, file.path(curr_out, "af.dimRed.rds"))
    #print(head(stringr::str_split(colnames(afin), "_")[]))
    condition <- stringr::str_split(colnames(afin), "_", simplify=T)[,2]
    print('condition')
    print(head(condition))
    cell_clusters <- data.frame(ID=colnames(afin), lineage=cluster_name, donor=donor, condition=condition)
    write.table(cell_clusters, file.path(curr_out, "cells_meta.tsv"), row.names=F, sep="\t", quote=F)
    out <- plot_af_clusters(afin, cluster_name)
    melt_df2 <- out[[1]]
    write.table(melt_df2, file.path(curr_out, "var_clusters.tsv"), row.names=F, sep="\t", quote=F)
    return(out)
}

# Run workflow

In [13]:
#colnames(SE) <- lapply(colnames(SE), function(x) paste0(x, name))
print("clones w all cells")
#curr_SE <- SE #filt_vars(SE, cells_f, vars_f)
out <- run_clones(SE, outdir, k.param=kparam)

[1] "clones w all cells"
[1] "afin"
       AAACGAAAGTCGTGAG-1_Control AAACGAACACAATAAG-1_Control
10086G                  0.2222222                  0.0000000
10318C                  0.0000000                  0.0000000
10373A                  0.0000000                  0.0000000
10398G                  1.0000000                  0.9534884
103A                    0.0000000                  0.0000000
10400T                  1.0000000                  0.9523810
       AAACGAAGTAAGTCTC-1_Control AAACGAAGTGGATTCT-1_Control
10086G                          0                          0
10318C                          0                          0
10373A                          0                          0
10398G                          1                          1
103A                            0                          0
10400T                          1                          1
       AAACGAATCGCAACTA-1_Control AAACTCGCAAGACTTC-1_Control
10086G                 0.06060606                

Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 4205
Number of edges: 2390923

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.0569
Number of communities: 1714
Elapsed time: 5 seconds


1658 singletons identified. 56 final clusters.



[1] "condition"
[1] "Control" "Control" "Control" "Control" "Control" "Control"


Saving 6.67 x 6.67 in image



[1] 147
[1] 45
[1] "Number of variants"
[1] 147
[1] "Number of clusters"
[1] 45


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



In [14]:
outdir

[1] "/home/isshamie/data/Projects/Mito_Trace/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_10/donor0"